In [1]:
import pandas as pd

file_name = "武汉专利协同次数统计.xlsx"
df = pd.read_excel(file_name)

In [2]:
df.head()

,公司1,公司2,次数
0,格力电器(武汉)有限公司,珠海格力电器股份有限公司,1419
1,鸿富锦精密工业(武汉)有限公司,鸿海精密工业股份有限公司,1193
2,中核武汉核电运行技术股份有限公司,核动力运行研究所,1043
3,美的集团武汉制冷设备有限公司,美的集团股份有限公司,821
4,武汉市黄鹤楼科技园有限公司,湖北中烟工业有限责任公司,561


## 连接mysql

In [3]:
import os
from tqdm import tqdm
import sys
import pymysql


sys.path.append("/home/jie/.key")
# 本地密码存储文件
from sql_key import password

In [4]:
# 连接到MySQL数据库
connection = pymysql.connect(
    host="localhost",  # MySQL数据库的主机
    user="root",  # MySQL用户名
    password=password,  # MySQL密码
    database="industry",  # 你要插入数据的数据库
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor,
)

In [5]:
new_data = {
    "公司1":[],
    "公司2":[],
    
    "公司1_经度":[],
    "公司1_纬度":[],
    
    "公司2_经度":[],
    "公司2_纬度":[],
    
    "次数":[],
}

In [19]:
# print(query_data(connection, "珠海格力电器股份有限公司"))

{'company_name': '珠海格力电器股份有限公司', 'longitude': Decimal('113.5502328'), 'latitude': Decimal('22.1337738')}


In [6]:
def query_data(connection, company_name):
    # 创建一个游标对象
    with connection.cursor() as cursor:
        # SQL 查询语句
        sql = """
        SELECT company_name, longitude, latitude
        FROM companies
        WHERE company_name = %s
        """
        # 执行查询
        cursor.execute(sql, (company_name,))
        res = cursor.fetchone()
        if res is None:
            return None, None, None
        # 获取查询结果
        return res["company_name"], res["longitude"], res["latitude"]

In [7]:
for _, row in tqdm(df.iterrows(), total=len(df)):
    com1, com2 = row["公司1"], row["公司2"]
    _, com1_longitude, com1_latitude = query_data(connection, com1)
    _, com2_longitude, com2_latitude = query_data(connection, com2)
    new_data["公司1"].append(com1)
    new_data["公司2"].append(com2)
    
    new_data["公司1_经度"].append(com1_longitude)
    new_data["公司1_纬度"].append(com1_latitude)
    
    new_data["公司2_经度"].append(com2_longitude)
    new_data["公司2_纬度"].append(com2_latitude)
    new_data["次数"].append(row["次数"])

100%|██████████| 8642/8642 [00:04<00:00, 1777.89it/s]


In [8]:
new_df = pd.DataFrame(new_data)
new_df.to_excel("武汉专利协同经纬度.xlsx")

In [30]:
# new_df.to_csv("武汉专利协同经纬度.csv")